In [ ]:
import markdown
import re
from markdown.util import etree
from markdown import Extension
from markdown.blockprocessors import BlockProcessor

In [ ]:
class BoxBlockProcessor(BlockProcessor):
    RE_FENCE_START = r"{: *([\s\S]*?)}([\s\S]*?)"  # start line, e.g., `   !!!! `
    RE_FENCE_END = r"{ *: *}"  # last non-blank line, e.g, '!!!\n  \n\n'

    def test(self, parent, block):
        return re.match(self.RE_FENCE_START, block)

    def run(self, parent, blocks):
        original_block = blocks[0]
        blocks[0] = re.sub(self.RE_FENCE_START, "", blocks[0])
        match = re.match(self.RE_FENCE_START, original_block)
        style = match.group(1)
        print(style)
        content = match.group(0)
        print(">", content)

        # Find block with ending fence
        for block_num, block in enumerate(blocks):
            if re.search(self.RE_FENCE_END, block):
                # remove fence
                blocks[block_num] = re.sub(self.RE_FENCE_END, "", block)
                # render fenced area inside a new div
                e = etree.SubElement(parent, "div")
                e.set("style", style)
                self.parser.parseBlocks(e, blocks[0 : block_num + 1])
                # remove used blocks
                for i in range(0, block_num + 1):
                    blocks.pop(0)
                return True  # or could have had no return statement
        # No closing marker!  Restore and do nothing
        blocks[0] = original_block
        return False  # equivalent to our test() routine returning False


class BoxExtension(Extension):
    def extendMarkdown(self, md):
        md.parser.blockprocessors.register(BoxBlockProcessor(md.parser), "box", 175)

In [ ]:
text = """
{:










kululu
shhf
asfasf
afafafsa
asfsga
}
fwfwfwf
{:}
"""

md = markdown.Markdown(extensions=[BoxExtension()])
html = md.convert(text)

print(html)